In [1]:
%pip install python-telegram-bot openai langchain-community docx2txt chromadb sentence-transformers tqdm pandas torch transformers nest_asyncio


You should consider upgrading via the '/Users/hq-w7n17ktq29/cursor/tg_dummy/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import zipfile
import logging
import csv
from datetime import datetime

from telegram import Update, ReplyKeyboardMarkup, KeyboardButton
from telegram.ext import (
    Application,
    CommandHandler,
    MessageHandler,
    filters,
    ContextTypes,
    ConversationHandler,
)
from openai import OpenAI
import nest_asyncio

from langchain.retrievers import ParentDocumentRetriever
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.stores import InMemoryStore
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.document_loaders import Docx2txtLoader

print("Все библиотеки успешно импортированы!")


/Users/hq-w7n17ktq29/cursor/tg_dummy/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Все библиотеки успешно импортированы!


In [3]:
# Конфигурация - ЗАМЕНИТЕ НА ВАШИ РЕАЛЬНЫЕ КЛЮЧИ!
TELEGRAM_BOT_TOKEN = "YOUR_TELEGRAM_BOT_TOKEN"
OPENAI_API_KEY = "YOUR_NEBIUS_AI_API_KEY"
BASE_URL = "https://api.studio.nebius.com/v1/"

# Подготовка данных
data_path = 'laws_docx'
zip_file_path = 'laws_docx.zip'

def unzip_if_needed(zip_path, extract_to):
    """Распаковывает архив, если целевая директория не существует."""
    if not os.path.exists(extract_to):
        print(f"Директория '{extract_to}' не найдена. Распаковываю '{zip_path}'...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall('.')
        print("Распаковка завершена.")
    else:
        print(f"Директория '{extract_to}' уже существует. Пропускаю распаковку.")

unzip_if_needed(zip_file_path, data_path)
print("Конфигурация завершена.")


Директория 'laws_docx' уже существует. Пропускаю распаковку.
Конфигурация завершена.


In [4]:
def get_docs(data_path):
    """Загружает документы .docx из указанной директории."""
    docs = []
    files = os.listdir(data_path)
    for file in files:
        if file.endswith('.docx'):
            loader = Docx2txtLoader(os.path.join(data_path, file))
            doc = loader.load()
            docs.append(doc[0])
    return docs

def create_retriever(docs, retriever_name='retriever', k=7, child_chunk_size=600, parent_chunk_size=3000):
    """Создает и настраивает ParentDocumentRetriever."""
    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=parent_chunk_size)
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=child_chunk_size)
    embedding_function = SentenceTransformerEmbeddings(model_name="sergeyzh/BERTA")

    print("Предупреждение: Используется хранилище в памяти. Эмбеддинги не будут сохранены.")
    store = InMemoryStore()
    
    vectorstore = Chroma(
        collection_name=retriever_name,
        embedding_function=embedding_function
    )

    retriever = ParentDocumentRetriever(
        vectorstore=vectorstore,
        docstore=store,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter,
        search_kwargs={"k": k}
    )
    
    print("Добавление документов в хранилище. Это может занять некоторое время...")
    for doc in docs:
        retriever.add_documents([doc])
    print("Документы успешно добавлены.")

    return retriever

def get_answer_with_llm(question, retriever, llm_client):
    """Получает ответ от LLM на основе вопроса и найденного контекста."""
    retrieved_docs = retriever.get_relevant_documents(question)
    context = " ".join([current_text.page_content for current_text in retrieved_docs])

    prompt = f"""Ответь на вопрос основываясь только на контексте.
    ВОПРОС: {question}
    КОНТЕКСТ: {context}
    """

    completion = llm_client.chat.completions.create(
        model="Qwen/Qwen3-32B-fast",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    raw_result = completion.choices[0].message.content

    # Убираем блок с размышлениями, если он есть
    if "</think>" in raw_result:
        result = raw_result.split("</think>", 1)[1].strip()
    else:
        result = raw_result.strip()

    return result

print("Функции для работы с документами и RAG определены.")


Функции для работы с документами и RAG определены.


In [5]:
# Настройка для работы в Jupyter
nest_asyncio.apply()

logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

# Состояния диалога
(
    SELECTING_USER_TYPE,
    SELECTING_TOPIC,
    RAG_QUESTION,
    FEEDBACK,
) = range(4)

# Кнопки интерфейса
USER_TYPE_BUTTONS = [["Я физ. лицо", "Я юр. лицо"]]
TOPIC_BUTTONS = [
    ["Передать показания", "Оплатить услуги"],
    ["Отключения", "Заключить договор"],
    ["Подключиться к сетям", "Тарифы"],
    ["Регистрация личного кабинета", "Прибор учета"],
    ["Свой вопрос"],
    ["Назад"],
]
FEEDBACK_BUTTONS = [["👍", "👎"]]

# Предопределенные ответы для физ. лиц (сокращенная версия для демонстрации)
PHYS_ANSWERS = {
    "Передать показания": "Срок передачи показаний с 1-го по 25-е число месяца. Передайте показания на сайте, в личном кабинете, мобильном приложении или через чат-бот Телеграм.",
    "Оплатить услуги": "Оплатите услуги на сайте, через личный кабинет, мобильное приложение, банковские сервисы или подключите автоплатеж.",
    "Отключения": "Информацию об отключениях смотрите на сайте. Сообщить об аварии можно через личный кабинет.",
    "Заключить договор": "Направьте документы через сайт или мобильное приложение.",
    "Подключиться к сетям": "Список документов доступен на сайте. Заявку можно подать через личный кабинет.",
    "Тарифы": "Информация о тарифах доступна на официальном сайте компании.",
    "Регистрация личного кабинета": "Используйте номер лицевого счета как логин и фамилию как пароль при первом входе.",
    "Прибор учета": "Информация об установке и замене приборов учета доступна на сайте."
}

JUR_ANSWERS = {
    "Передать показания": "Передавайте показания через личный кабинет для бизнеса или бизнес-бот в Telegram.",
    "Оплатить услуги": "Оплачивайте через личный кабинет для бизнеса, мобильное приложение или банковские сервисы.",
    "Отключения": "Информация об отключениях на сайте. Сообщить об аварии: 8-800-100-9777.",
    "Заключить договор": "Направьте документы через сайт или бизнес-приложение.",
    "Подключиться к сетям": "Документы и заявки через сайт или личный кабинет ИЭСК.",
    "Тарифы": "Информация о тарифах и ценах доступна на сайте.",
    "Регистрация личного кабинета": "Используйте номер договора как логин и ИНН как пароль.",
    "Прибор учета": "Информация о приборах учета для бизнеса доступна на сайте."
}

def log_user_interaction(user_id, question, answer=None):
    """Логирует взаимодействия пользователя в CSV файл."""
    file_exists = os.path.isfile("user_interactions.csv")
    with open("user_interactions.csv", "a", newline="", encoding="utf-8") as csvfile:
        fieldnames = ["timestamp", "user_id", "question", "answer"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        
        row = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "user_id": user_id,
            "question": question,
            "answer": answer if answer else "N/A"
        }
        writer.writerow(row)

print("Константы и вспомогательные функции бота определены.")


Константы и вспомогательные функции бота определены.


In [6]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Начинает диалог и спрашивает тип пользователя."""
    reply_markup = ReplyKeyboardMarkup(USER_TYPE_BUTTONS, one_time_keyboard=True, resize_keyboard=True)
    await update.message.reply_text(
        "Здравствуйте! Я робот-консультант «Иркутскэнергосбыт».\nПожалуйста, выберите ваш статус:",
        reply_markup=reply_markup,
    )
    return SELECTING_USER_TYPE

async def select_user_type(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Сохраняет тип пользователя и предлагает выбрать тему."""
    context.user_data['user_type'] = update.message.text
    reply_markup = ReplyKeyboardMarkup(TOPIC_BUTTONS, resize_keyboard=True)
    await update.message.reply_text("Выберите тему Вашего вопроса:", reply_markup=reply_markup)
    return SELECTING_TOPIC

async def handle_topic(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Обрабатывает выбор темы."""
    topic = update.message.text
    user_type = context.user_data.get('user_type')

    if topic == "Назад":
        return await start(update, context)

    if topic == "Свой вопрос":
        await update.message.reply_text("Пожалуйста, введите ваш вопрос текстом:")
        return RAG_QUESTION

    answers = PHYS_ANSWERS if user_type == "Я физ. лицо" else JUR_ANSWERS
    answer = answers.get(topic, "Пожалуйста, выберите одну из предложенных тем.")
    
    await update.message.reply_text(answer)
    log_user_interaction(update.message.from_user.id, topic, answer)
    
    reply_markup = ReplyKeyboardMarkup(FEEDBACK_BUTTONS, one_time_keyboard=True, resize_keyboard=True)
    await update.message.reply_text("Оцените ответ бота", reply_markup=reply_markup)
    return FEEDBACK

async def handle_rag_question(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Обрабатывает произвольный вопрос с помощью RAG."""
    question = update.message.text
    retriever = context.bot_data["retriever"]
    llm_client = context.bot_data["llm_client"]

    await update.message.reply_text("Ваш вопрос обрабатывается, пожалуйста, подождите...")

    try:
        answer = get_answer_with_llm(question, retriever, llm_client)
        log_user_interaction(update.message.from_user.id, question, answer)
        if len(answer) > 4000:
            for i in range(0, len(answer), 4000):
                await update.message.reply_text(answer[i:i+4000])
        else:
            await update.message.reply_text(answer)
    except Exception as e:
        logger.error(f"Error processing RAG question: {e}")
        await update.message.reply_text("Извините, произошла ошибка при обработке вашего вопроса.")

    reply_markup = ReplyKeyboardMarkup(FEEDBACK_BUTTONS, one_time_keyboard=True, resize_keyboard=True)
    await update.message.reply_text("Оцените ответ бота", reply_markup=reply_markup)
    return FEEDBACK
    
async def handle_feedback(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Обрабатывает обратную связь от пользователя."""
    feedback = update.message.text
    log_user_interaction(update.message.from_user.id, "feedback", feedback)
    
    reply_markup = ReplyKeyboardMarkup(TOPIC_BUTTONS, resize_keyboard=True)
    await update.message.reply_text("Спасибо за вашу оценку! Выберите следующую тему:", reply_markup=reply_markup)
    return SELECTING_TOPIC

async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Отменяет и завершает диалог."""
    await update.message.reply_text("Диалог завершен. Для нового вопроса нажмите /start")
    return ConversationHandler.END

def setup_bot(retriever):
    """Настраивает и запускает Telegram-бота."""
    application = Application.builder().token(TELEGRAM_BOT_TOKEN).build()
    llm_client = OpenAI(api_key=OPENAI_API_KEY, base_url=BASE_URL)

    application.bot_data["retriever"] = retriever
    application.bot_data["llm_client"] = llm_client

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler("start", start)],
        states={
            SELECTING_USER_TYPE: [MessageHandler(filters.Regex(r'^(Я физ\. лицо|Я юр\. лицо)$'), select_user_type)],
            SELECTING_TOPIC: [MessageHandler(filters.TEXT & ~filters.COMMAND, handle_topic)],
            RAG_QUESTION: [MessageHandler(filters.TEXT & ~filters.COMMAND, handle_rag_question)],
            FEEDBACK: [MessageHandler(filters.Regex(r'^(👍|👎)$'), handle_feedback)],
        },
        fallbacks=[CommandHandler("cancel", cancel)],
    )

    application.add_handler(conv_handler)
    print("Бот настроен и запущен! Для остановки используйте Kernel -> Interrupt в меню.")
    application.run_polling()

print("Обработчики событий и функция настройки бота определены.")


Обработчики событий и функция настройки бота определены.


In [ ]:
# Главная функция запуска
def main():
    """Главная функция для запуска юридического помощника."""
    
    print("Загрузка документов...")
    docs = get_docs(data_path)
    print(f"Загружено {len(docs)} документов.")

    print("Создание ретривера...")
    retriever = create_retriever(docs)

    print("Запуск бота...")
    setup_bot(retriever)

# ЗАПУСК БОТА
if __name__ == "__main__":
    main()


Загрузка документов...
Загружено 6 документов.
Создание ретривера...


/var/folders/1b/1vpr3vf56sjc44fq2x4hnmv40000gn/T/ipykernel_68551/2944534551.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="sergeyzh/BERTA")
/Users/hq-w7n17ktq29/cursor/tg_dummy/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-11 10:21:26,198 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: mps
2025-07-11 10:21:26,198 - sentence_transformers.SentenceTransformer - I

Предупреждение: Используется хранилище в памяти. Эмбеддинги не будут сохранены.


2025-07-11 10:21:31,168 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Добавление документов в хранилище. Это может занять некоторое время...
Документы успешно добавлены.
Запуск бота...
Бот настроен и запущен! Для остановки используйте Kernel -> Interrupt в меню.


2025-07-11 10:23:21,541 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8013234056:AAHmNKzV8UuWYTaUSmWoQO5K7C-I4UXhtQs/getMe "HTTP/1.1 200 OK"
2025-07-11 10:23:21,646 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8013234056:AAHmNKzV8UuWYTaUSmWoQO5K7C-I4UXhtQs/deleteWebhook "HTTP/1.1 200 OK"
2025-07-11 10:23:21,649 - telegram.ext.Application - INFO - Application started
2025-07-11 10:23:32,162 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8013234056:AAHmNKzV8UuWYTaUSmWoQO5K7C-I4UXhtQs/getUpdates "HTTP/1.1 200 OK"
2025-07-11 10:23:39,966 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8013234056:AAHmNKzV8UuWYTaUSmWoQO5K7C-I4UXhtQs/getUpdates "HTTP/1.1 200 OK"
2025-07-11 10:23:40,364 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8013234056:AAHmNKzV8UuWYTaUSmWoQO5K7C-I4UXhtQs/sendMessage "HTTP/1.1 200 OK"
2025-07-11 10:23:41,239 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8013234056:AA